In [1]:
import json

In [2]:
from pyspark.sql import SparkSession

In [3]:
config = json.load(open("config.json"))

In [4]:
spark = SparkSession.builder\
    .config("spark.jars", config['JDBC_PATH']) \
    .config("spark.executor.extraClassPath", config['JDBC_PATH'])\
    .getOrCreate()

24/01/09 01:02:59 WARN Utils: Your hostname, przemyslaw-Latitude-E7450 resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
24/01/09 01:02:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/09 01:03:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/09 01:03:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
dbtable = "inventory_equipment"

In [6]:
df_inventory_equipment = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [7]:
df_inventory_equipment.createOrReplaceTempView(dbtable)

In [9]:
dbtable = "dim_equipment"

In [10]:
df_equipment = spark.read.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .load()

In [11]:
df_equipment.createOrReplaceTempView(dbtable)

In [12]:
dbtable = "dim_inventory"

In [13]:
df_inventory = spark.read.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .load()

In [14]:
df_inventory.createOrReplaceTempView(dbtable)

In [15]:
bridge_inventory_equipment = spark.sql("""
SELECT
i.inventory_id AS inventory_id,
e.equipment_id AS equipment_id
FROM inventory_equipment AS ie
JOIN dim_inventory AS i ON ie.inventory_id = i.inventory_key
JOIN dim_equipment AS e ON ie.equipment_id = e.equipment_key
""")

In [16]:
bridge_inventory_equipment.show()

+------------+------------+
|inventory_id|equipment_id|
+------------+------------+
|         964|           1|
|         191|           1|
|        1010|           1|
|         541|           1|
|          65|           1|
|         558|           1|
|        1127|           1|
|         293|           1|
|         270|           1|
|         222|           1|
|         730|           1|
|        1145|           1|
|         243|           1|
|         705|           1|
|         278|           1|
|         367|           1|
|         847|           1|
|         277|           1|
|         415|           1|
|         287|           1|
+------------+------------+
only showing top 20 rows



In [ ]:
spark.sql("""
ALTER TABLE bridge_inventory_equipment ADD PRIMARY KEY (inventory_id, equipment_id)
ALTER TABLE dim_equipment ADD PRIMARY KEY (equipment_id)

ALTER TABLE bridge_inventory_equipment
ADD CONSTRAINT bridge_inventory_equipment_FK
FOREIGN KEY (inventory_id) REFERENCES dim_inventory (inventory_id)

ALTER TABLE bridge_inventory_equipment
ADD CONSTRAINT bridge_inventory_equipment_FK2
FOREIGN KEY (equipment_id) REFERENCES dim_equipment (equipment_id)
""")

In [17]:
bridge_inventory_equipment.write.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", "bridge_inventory_equipment")\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .save()